In [ ]:
# import required libraries
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import os
import re
from pathlib import Path

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op
from typing import NamedTuple

In [ ]:
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
REGION = "northamerica-northeast1"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}"

In [ ]:
@component(base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest", output_component_file="component_one.yaml")
def input_integer(num: int) -> int:
    return num

In [ ]:
@component(base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest", output_component_file="component_two.yaml")
def double_square(
    num: int
) -> NamedTuple(
    "Outputs",
    [
        ("value", int),  # Return parameters
        ("double", int),
        ("square", int)
    ],
):
    double = num * 2 
    square = num * num

    print(f"input value: {num}, double: {double}, square: {square}") 
    return (num, double, square)

In [ ]:
@component(base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest", output_component_file="component_three.yaml")
def show_results(
    num: int,
    double: int,
    square: int
) -> str:
    print("Here is the output: ")

    end_str = f"The double of {num} is {double}, and the square of {num} is {square}"

    return end_str

In [ ]:
@dsl.pipeline(
    name="demo-pipeline",
    description="vertex pipeline example",
    pipeline_root=PIPELINE_ROOT,
)
# You can change the `text` and `emoji_str` parameters here to update the pipeline output
def pipeline(num: int = 5):
    
    # ----- component 1 --------
    input_integer_op = input_integer(num)
    
    # ----- component 2 --------
    double_square_op = double_square(num)
    
    # ----- component 3 --------
    show_results_op = show_results(
            input_integer_op.output,
            double_square_op.outputs["double"],
            double_square_op.outputs["square"] ,
        ) 
    
    show_results_op.after(input_integer_op)
    show_results_op.after(double_square_op)


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name="demo-pipeline-job",
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = PIPELINE_ROOT,
   location=REGION,
   enable_caching=True # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()

In [ ]:
# @dsl.pipeline(
#     name="hello-world",
#     description="An intro pipeline",
#     pipeline_root=PIPELINE_ROOT,
# )
# # You can change the `text` and `emoji_str` parameters here to update the pipeline output
# def intro_pipeline(text: str = "Vertex Pipelines", emoji_str: str = "sparkles"):
#     product_task = product_name(text)
#     emoji_task = emoji(emoji_str)
#     consumer_task = build_sentence(
#         product_task.output,
#         emoji_task.outputs["emoji"],
#         emoji_task.outputs["emoji_text"],
#     )

In [ ]:
def double_square(
    num: int,
) -> NamedTuple(
    "Outputs",
    [
        ("value", int),  # Return parameters
        ("double", int),
        ("square", int)
    ],
):
    double = num * 2 
    square = num * num

    print(f"input value: {num}, double: {double}, square: {square}") 
    return (num, double, square)

In [ ]:
double_square(5)

## MLOps With Kubeflow Pipelines (Part 1)

### 1) Data Extraction

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# import the entire dataset into `data` for quick EDA
data = load_breast_cancer() 
df = pd.DataFrame(data = data.data, columns = data.feature_names) 
df['target'] = pd.Series(data.target) 

#import features into X and target into y for training
X, y = load_breast_cancer(return_X_y = True, as_frame = True) 

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

df.head()

In [ ]:
df.groupby('target')[['mean radius', 'mean perimeter', 'mean area']].mean()

In [ ]:
df['target'].value_counts()

In [ ]:
# Check target names
print(data.target_names)

# Check value counts
print(df['target'].value_counts()) 

# Analyzing the target variable
plt.title('Count of cancer type')
sns.countplot(x=df['target'])
plt.xlabel('Cancer Severity')
plt.ylabel('Count')
plt.show()

In [ ]:
# Plotting correlation between diagnosis and radius
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
sns.boxplot(x="target", y="mean radius", data=df)
plt.subplot(1,3,2)
sns.boxplot(x="target", y="mean perimeter", data=df)
plt.subplot(1,3,3)
sns.boxplot(x="target", y="mean concave points", data=df)
plt.show()

In [ ]:
df.columns

In [ ]:
# Import Matplotlib and Seaborn
import matplotlib.pyplot as plt
import seaborn as sns

# Create scatter plot of horsepower vs. mpg
sns.relplot(x='mean radius', y='mean concave points', data=df, style='target', hue='target', kind='scatter')

# Show plot
plt.show()

In [ ]:
# Size the plot
plt.figure(figsize=(15,4))

# Plotting bivariate relations between each pair of features 
sns.pairplot(df, hue="target", vars = ["mean radius", "mean concave points", "mean texture", "mean smoothness"])

# Show plot
plt.show()

In [ ]:
# Viewing the data statistics
df.describe()

### 2) Model Training

### Import Libraries

In [ ]:
# Import Libraries
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
from google.cloud.aiplatform import pipeline_jobs
from typing import NamedTuple
import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op
import pandas as pd
import numpy as np


### Import Data

In [ ]:

# Import Dataset and Save in GCS
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="import_data.yaml",
)
def import_data(
            dataset_id: str,
            file_bucket: str, 
    ) -> NamedTuple(
        "Outputs", 
        [
         ("save_path", str), 
         ("col_list", list)
        ]
    ):
    # Import Libraries
    import pandas as pd
    import numpy as np
    from sklearn.datasets import load_breast_cancer

    # import the entire dataset into 'data'
    data = load_breast_cancer() 
    
    # save the data in df, including the targets
    df = pd.DataFrame(data = data.data, columns = data.feature_names) 
    df['target'] = pd.Series(data.target) 
    
    # save df in cloud storage 
    save_path = f'gs://{file_bucket}/{dataset_id}/{dataset_id}_data.csv'
    df.to_csv(save_path, index=True) 
    
    print(f'{dataset_id}_data.csv saved in {save_path}')
    
    col_list = list([col for col in df.columns if col != "target"])
    
    return (save_path, col_list)


### Model Training

In [ ]:

# Load Dataset and Train Model
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="train_model.yaml",
)
def model_training(
            dataset_id: str,
            file_bucket: str, 
            save_path: str,
            model: Output[Model],
            metrics: Output[Metrics],
            metricsc: Output[ClassificationMetrics], 
            col_list: list 
    ) -> NamedTuple(
        "Outputs",
        [
            ("accuracy", float),  # Return parameters
            ("f1_score", float),
            ("roc_auc", float), 
            ("X_y_val_index", list), 
            ("model_location", str)
        ],
    ):
    # Import Libraries
    import gc
    import time
    from datetime import datetime
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import xgboost as xgb
    import pickle
    import logging
    from google.cloud import storage
    from google.cloud import bigquery
    from sklearn.datasets import load_breast_cancer
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
    
    # Read csv that was saved in 'import_data' component
    df = pd.read_csv(save_path)  

    # X and y
    y = np.squeeze(df['target'].values)
    X = df.drop(columns='target')
    
    # Create the training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    
    # Reserve some samples for final validation
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=123)

    # Instantiate the XGB Classifier: xgb_model
    xgb_model = xgb.XGBClassifier(
        learning_rate=0.01,
        n_estimators=100,
        max_depth=8,
        min_child_weight=1,
        max_delta_step=1, 
        colsample_bytree=0.9,
        subsample=0.9,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1, 
        eval_metric='auc', 
        base_score=0.5
    )

    # Fit the classifier to the training set
    xgb_model.fit(X_train, y_train)
    
    # Predict based on X_test
    y_pred = xgb_model.predict(X_test)
    y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
    
    # Model accuracy 
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    # Precision & Recall 
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # F1 Score 
    f1_score = f1_score(y_test, y_pred)
    print("F1 Score:", f1_score)

    # ROC AUC Score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print("ROC AUC Score:", roc_auc)

    # Log eval metrics
    metrics.log_metric("Model", "XGBClassifier")
    metrics.log_metric("Size", df.shape[0])
    metrics.log_metric("Accuracy",accuracy)
    metrics.log_metric("AUC", roc_auc)
    metrics.log_metric("Precision", precision) 
    metrics.log_metric("Recall", recall) 
    metrics.log_metric("F1_Score", f1_score)

    # Compute fpr, tpr, thresholds for the ROC Curve
    fpr, tpr, thresholds = roc_curve(
        y_true=y_test, y_score=y_pred_proba, pos_label=True
    )
    
    # Log classification metrics
    metricsc.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())
    metricsc.log_confusion_matrix(['Malignant', 'Benign'], confusion_matrix(y_test, y_pred).tolist())

    # added to model_training component: save model artifacts in GCS bucket
    model_artifacts = {}
    create_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    model_artifacts['create_time'] = create_time
    model_artifacts['model'] = xgb_model
    model_artifacts['col_list'] = col_list
    
    # create and write model_artifacts.pkl
    with open('model_artifacts.pkl', 'wb') as pkl_file:
        pickle.dump(model_artifacts, pkl_file)

        # Use the 'pickle.dump()' method to serialize and store the 'model_artifacts' data
        pickle.dump(model_artifacts, pkl_file)

    # create a gcs bucket instance
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(file_bucket)
    
    # define the folder path where the models will be saved. create one if not found. 
    model_path = 'breast_cancer_models/'
    blob = bucket.blob(model_path)
    if not blob.exists(storage_client):
        blob.upload_from_string('')
    
    # set model name and upload 'model_artifacts.pkl' to the folder in gcs bucket 
    model_name = 'breast_cancer_models_{}'.format(model_artifacts['create_time'])
    model_location = f'{model_path}{model_name}'
    blob = bucket.blob(model_location)
    blob.upload_from_filename('model_artifacts.pkl')
    
    print(f"Model artifacts loaded to GCS Bucket: {model_location}")
    
#     model.metadata['accuracy'] = accuracy
#     model.metadata['precision'] = precision
#     model.metadata['recall'] = recall
#     model.metadata['f1_score'] = f1_score
#     model.metadata['auc'] = roc_auc
    
    model.uri = f'gs://{file_bucket}/{model_location}'
    
#     # Log additional model details 
#     with open(model.path, 'w') as output_file:
#         output_file.write(f'You can enter additional model details here')
#     output_file.close()
    
    time.sleep(120)

    return (accuracy, f1_score, roc_auc, list(X_val.index), model_location)
    

### Model Evaluation

In [ ]:
# Evaluate the model performance
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="model_evaluation.yaml",
)
def model_evaluation(
            accuracy: float, 
            f1_score: float, 
            roc_auc: float, 
            accuracy_threshold: float, 
            f1_score_threshold: float, 
            roc_auc_threshold: float
            ) -> NamedTuple(
                "Output", [("result", str)]
            ):
    
    # Set checker to True
    checker = True
    
    # Set checker to False if any of the eval metrics is below threshold
    if accuracy < accuracy_threshold: 
        checker = False 
    if f1_score < f1_score_threshold: 
        checker = False 
    if roc_auc < roc_auc_threshold: 
        checker = False 
        
    # if checker == True, return "Pass", otherwise return "Fail"
    if checker == True: 
        return ("Pass",) 
    else: 
        return ("Fail",)
    

### Upload model to Model Registry

In [ ]:
# import required libraries
import kfp
from kfp import dsl
from kfp.v2.dsl import (Model, Input, Output, component)

# Component for uploading model to Vertex Model Registry
@component(
# Uploads model
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="model-upload.yaml",
)

def upload_model_to_mr(
    project_id: str,
    model: Input[Model],
    vertex_model: Output[Model],
    region: str,
    model_name: str,
    prediction_image: str,
    col_list: list, 
    result: str
):
    """
    Upload model to Vertex Model Registry.
    Args:
        project_id (str): project id for where this pipeline is being run
        model (Input[Model]): model passed in from training component. Must have path specified in model.uri
        region (str): region for where the query will be run
        model_name (str): name of model to be stored
        prediction_image (str): prediction image uri
        col_list (str): string of list of columns in serving data
    Returns:
        vertex_model (Output[Model]): Model saved in Vertex AI
    """

    from google.cloud import aiplatform
    import os
    from datetime import datetime

    aiplatform.init(project=project_id, location=region)
    
    ## check if prediction image is custom or not
    if prediction_image.startswith('northamerica-northeast1-docker'):
        # custom: must set ports
        health_route = "/ping"
        predict_route = "/predict"
        serving_container_ports = [7080]
    else:
        # Google pre-built
        health_route = None
        predict_route = None
        serving_container_ports = None

    if result == "Pass": 

        ## check for existing models
        # if model exists, update the version
        try:
            model_uid = aiplatform.Model.list(
                filter=f'display_name={model_name}', 
                order_by="update_time",
                location=region)[-1].resource_name

            uploaded_model = aiplatform.Model.upload(
                display_name = model_name, 
                artifact_uri = os.path.dirname(model.uri),
                serving_container_image_uri = prediction_image,
                serving_container_environment_variables =  {"COL_LIST":str(col_list)}, # remove for posting
                parent_model = model_uid,
                is_default_version = True
            )
        # if model does not already exist, create a new model
        except:
            uploaded_model = aiplatform.Model.upload(
                display_name = model_name,
                artifact_uri = os.path.dirname(model.uri),
                serving_container_image_uri=prediction_image,
                serving_container_environment_variables =  {"COL_LIST":str(col_list)}, # remove for posting
            )

        vertex_model.uri = uploaded_model.resource_name
        vertex_model.version_create_time = datetime.now()
        vertex_model.version_description = "breast cancer model" 
    
    else: 
        
        print("Training performance is not satisfactory. Upload to the Model Registry revoked.")
        
    

### Load Model to Notebook

In [ ]:
from kfp.v2.dsl import (Artifact, Output, Input, HTML, component)

# Load Custom Model Component: load in most recent version of your model to run batch predictions with
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="load_model.yaml"
)
# this model returns a model artifact that will be passed on to Batch Predictions
def load_model(
                project_id: str, 
                region: str, 
                model_name: str, 
                model: Output[Artifact]):
    
    from google.cloud import aiplatform
    
    model_uid = aiplatform.Model.list(
                                    filter=f'display_name={model_name}', 
                                    order_by="update_time",
                                    location=region)[-1].resource_name
    model.uri = model_uid
    model.metadata['resourceName'] = model_uid
    

### Batch Prediction - 1

In [ ]:
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component, HTML)
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="batch_prediction.yaml",
)
def batch_prediction(
        project_id: str,
        dataset_id: str,
        file_bucket: str,
        val_index: list, 
        save_path: str, 
        model: Input[Model],
        metrics: Output[Metrics],
        metricsc: Output[ClassificationMetrics],
):
    import time
    import pandas as pd
    import numpy as np
    import pickle
    from datetime import date
    from dateutil.relativedelta import relativedelta
    from google.cloud import bigquery
    from google.cloud import storage
    from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
    
    # Read csv that was saved in 'import_data' component
    df = pd.read_csv(save_path)  

    # X and y
    X = df.drop(columns='target')
    y = df['target']
    
    X_val = X.loc[val_index] 
    y_val = np.squeeze(y.iloc[val_index].values) 

    time.sleep(10)

    model_path = 'breast_cancer_models/'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(file_bucket)
    blobs = storage_client.list_blobs(file_bucket, prefix='{}breast_cancer_models'.format(model_path))

    model_lists = []
    for blob in blobs:
        model_lists.append(blob.name)

    blob = bucket.blob(model_lists[-1])
    blob_in = blob.download_as_string()
    model_dict = pickle.loads(blob_in)
    model_xgb = model_dict['model']
    features = model_dict['col_list']
    print('...... model loaded')
    time.sleep(10)

    # get full score to cave into bucket
    y_pred = model_xgb.predict(X_val)
    y_pred_proba = model_xgb.predict_proba(X_val)[:, 1] 
    
    result = pd.DataFrame(columns=['index', 'y_pred_proba', 'y_pred', 'y_val'])
    result['index'] = pd.Series(X_val.index.to_list())
    # result['index'] = result['index'].astype('int64')
    result['y_pred_proba'] = y_pred_proba
    # result['y_pred_proba'] = result['y_pred_proba'].fillna(0.0).astype('float64')
    result['y_pred'] = y_pred
    result['y_test'] = y_val

    result.to_csv('gs://{}/breast_cancer/model_validation.csv'.format(file_bucket), index=True)

    # Model accuracy 
    accuracy = accuracy_score(y_val, y_pred)
    print("Accuracy:", accuracy)
    
    # Precision & Recall 
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    
    # F1 Score 
    f1_score = f1_score(y_val, y_pred)
    print("F1 Score:", f1_score)

    # ROC AUC Score
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    print("ROC AUC Score:", roc_auc)

    # Log eval metrics
    metrics.log_metric("Model", "XGBClassifier")
    metrics.log_metric("Size", X_val.shape[0])
    metrics.log_metric("Accuracy", accuracy)
    metrics.log_metric("AUC", roc_auc)
    metrics.log_metric("Precision", precision) 
    metrics.log_metric("Recall", recall) 
    metrics.log_metric("F1_Score", f1_score)
    
    time.sleep(60)
    print(f"Batch prediction for {X_val.shape[0]} samples completed")
    


### Batch Prediction - 2

In [ ]:
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component, HTML)
@component(
    base_image="northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest",
    output_component_file="batch_prediction.yaml",
)
def batch_prediction(
        region: str, 
        project_id: str,
        dataset_id: str,
        file_bucket: str,
        val_index: list, 
        save_path: str, 
        model_name: str, 
        model: Output[Artifact],
        metrics: Output[Metrics],
        metricsc: Output[ClassificationMetrics],
):

    import time
    import pandas as pd
    import numpy as np
    import pickle
    from datetime import date
    from dateutil.relativedelta import relativedelta
    from google.cloud import bigquery
    from google.cloud import storage
    from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
    from google.cloud import aiplatform
    
    model_uid = aiplatform.Model.list(
                                    filter=f'display_name={model_name}', 
                                    order_by="update_time",
                                    location=region)[-1].resource_name
    model.uri = model_uid
    model.metadata['resourceName'] = model_uid
    
    # Read csv that was saved in 'import_data' component
    df = pd.read_csv(save_path)  

    # X and y
    X = df.drop(columns='target')
    y = df['target']
    
    X_val = X.loc[val_index] 
    y_val = np.squeeze(y.iloc[val_index].values) 

    time.sleep(10)

    model_path = 'breast_cancer_models/'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(file_bucket)
    blobs = storage_client.list_blobs(file_bucket, prefix='{}breast_cancer_models'.format(model_path))

    model_lists = []
    for blob in blobs:
        model_lists.append(blob.name)

    blob = bucket.blob(model_lists[-1])
    blob_in = blob.download_as_string()
    model_dict = pickle.loads(blob_in)
    model_xgb = model_dict['model']
    features = model_dict['col_list']
    print('...... model loaded')
    time.sleep(10)

    # get full score to cave into bucket
    y_pred = model_xgb.predict(X_val)
    y_pred_proba = model_xgb.predict_proba(X_val)[:, 1] 
    
    result = pd.DataFrame(columns=['index', 'y_pred_proba', 'y_pred', 'y_val'])
    result['index'] = pd.Series(X_val.index.to_list())
    # result['index'] = result['index'].astype('int64')
    result['y_pred_proba'] = y_pred_proba
    # result['y_pred_proba'] = result['y_pred_proba'].fillna(0.0).astype('float64')
    result['y_pred'] = y_pred
    result['y_test'] = y_val

    result.to_csv('gs://{}/breast_cancer/model_validation.csv'.format(file_bucket), index=True)

    # Model accuracy 
    accuracy = accuracy_score(y_val, y_pred)
    print("Accuracy:", accuracy)
    
    # Precision & Recall 
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    
    # F1 Score 
    f1_score = f1_score(y_val, y_pred)
    print("F1 Score:", f1_score)

    # ROC AUC Score
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    print("ROC AUC Score:", roc_auc)

    # Log eval metrics
    metrics.log_metric("Model", "XGBClassifier")
    metrics.log_metric("Size", X_val.shape[0])
    metrics.log_metric("Accuracy", accuracy)
    metrics.log_metric("AUC", roc_auc)
    metrics.log_metric("Precision", precision) 
    metrics.log_metric("Recall", recall) 
    metrics.log_metric("F1_Score", f1_score)
    
    time.sleep(60)
    print(f"Batch prediction for {X_val.shape[0]} samples completed")
    


In [ ]:
# import time
# import pandas as pd
# import numpy as np
# import pickle
# from datetime import date
# from dateutil.relativedelta import relativedelta
# from google.cloud import bigquery
# from google.cloud import storage

# file_bucket = FILE_BUCKET
# dataset_id = DATASET_ID 

# save_path = f'gs://{file_bucket}/{dataset_id}/{dataset_id}_data.csv'
# val_index = [22, 138, 192, 190, 260, 498, 157, 309, 454, 166, 202, 488, 33, 480, 205, 345, 334, 175, 520, 399, 511, 24, 400, 49, 74, 230, 557, 327, 43, 436, 456, 287, 209, 410, 326]

# # Read csv that was saved in 'import_data' component
# df = pd.read_csv(save_path)  

# # X and y
# X = df.drop(columns='target')
# y = df['target']

# X_val = X.loc[val_index] 
# y_val = np.squeeze(y.iloc[val_index].values) 

# result = pd.DataFrame(columns=['index', 'y_pred_proba', 'y_pred', 'y_val'])
# result['index'] = X_val.index
# result['index'] = result['index'].astype(int)


# result

In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
DATASET_ID = 'breast_cancer'
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
MODEL_ID = '5070'
REGION = 'northamerica-northeast1'
MODEL_NAME = 'breast_cancer'
PREDICTION_IMAGE = 'northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/wb-platform/pipelines/kubeflow-pycaret:latest'

In [ ]:
# library imports
from kfp.v2 import compiler
from google.cloud.aiplatform import pipeline_jobs
@dsl.pipeline(
    name='breast-cancer-pipeline', 
    description='breast-cancer-pipeline'
    )
def pipeline(
        dataset_id: str = DATASET_ID, 
        file_bucket: str = FILE_BUCKET, 
        region: str = REGION
    ):
    
    import google.oauth2.credentials
    token = !gcloud auth print-access-token
    token_str = token[0]
    
    # ----- create training set --------
    import_data_op = import_data(dataset_id=dataset_id,
                          file_bucket=file_bucket)
    
    model_training_op = model_training(dataset_id=dataset_id,
                          file_bucket=file_bucket, 
                          save_path=import_data_op.outputs['save_path'], 
                          col_list = import_data_op.outputs["col_list"])
    
    model_evaluation_op=  model_evaluation(
                          accuracy=model_training_op.outputs["accuracy"], 
                          f1_score=model_training_op.outputs["f1_score"], 
                          roc_auc=model_training_op.outputs["roc_auc"], 
                          accuracy_threshold=0.95, 
                          f1_score_threshold=0.95, 
                          roc_auc_threshold=0.95
                          )
    
    upload_model_to_mr_op = upload_model_to_mr(
                        project_id = PROJECT_ID,
                        region = REGION,
                        model = model_training_op.outputs["model"],
                        model_name = MODEL_NAME,
                        prediction_image = PREDICTION_IMAGE,
                        col_list = import_data_op.outputs["col_list"], 
                        result = model_evaluation_op.outputs['result'])
    
    load_model_op = load_model(
                        project_id= PROJECT_ID, 
                        region= REGION, 
                        model_name= MODEL_NAME)
    
    batch_prediction_op = batch_prediction(
                        project_id = PROJECT_ID,
                        dataset_id = DATASET_ID,
                        file_bucket = FILE_BUCKET, 
                        val_index = model_training_op.outputs['X_y_val_index'], 
                        save_path = import_data_op.outputs['save_path'], 
                        model = load_model_op.output)

    model_training_op.after(import_data_op)
    model_evaluation_op.after(model_training_op)
    upload_model_to_mr_op.after(model_evaluation_op)
    load_model_op.after(upload_model_to_mr_op)
    batch_prediction_op.after(load_model_op)
    


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name='breast-cancer-pipeline',
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = f"gs://{FILE_BUCKET}",
   location=REGION,
   enable_caching=False # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()

In [ ]:
points_balance_to_redeem = {5:[0.00277532483915731,
0.145073798410495,
0.295193641983095,
0.379714898448341,
0.452125646524536,
0.789453765611202,
0.836634287876876,
0.915100290147597,
0.93414911063454,
0.94096127160338,
0.981455784029267,
0.983978806610319,
0.98788949161095,
0.988772549514318,
0.990412514192002,
0.992809385644001,
0.993061687902106,
0.993566292418317,
0.993692443547369,
0.994070896934527,
0.997476977418948,
0.997603128548,
0.997729279677053,
0.997855430806105,
0.998233884193263,
0.998360035322316,
0.998612337580421,
0.998864639838526,
0.999116942096631,
0.999243093225684,
0.999495395483789,
0.999621546612842,
0.999873848870947,
1,],
10:[0.00180658873538788,
0.0687566418703506,
0.18788522848034,
0.255047821466524,
0.312539851222104,
0.731562167906482,
0.777364505844845,
0.883846971307119,
0.90414452709883,
0.912646121147715,
0.982040382571732,
0.984803400637619,
0.990541976620616,
0.99096705632306,
0.991604675876726,
0.994261424017002,
0.994580233793836,
0.994686503719447,
0.994899043570669,
0.995111583421891,
0.998087141339,
0.998299681190222,
0.998724760892666,
0.998831030818277,
0.999043570669499,
0.999468650371944,
0.999574920297555,
0.999681190223166,
0.999787460148777,
0.999893730074388,
0.999999999999999,],
15:[0.000961076405574243,
0.039884670831331,
0.0896859027565418,
0.125595212092088,
0.17347429120615,
0.633436721855751,
0.676335677777292,
0.797562360753134,
0.819230265169717,
0.829059455681272,
0.973220916517408,
0.979162116115504,
0.989952383032633,
0.991612424096806,
0.992966668122843,
0.995718841466078,
0.99611200908654,
0.996592547289327,
0.99681097374514,
0.996898344327465,
0.99912629417675,
0.999169979467913,
0.9993010353414,
0.999606832379538,
0.9996505176707,
0.999781573544187,
0.99982525883535,
0.999912629417675,
0.999956314708837,
1,],
25:[0.000887193363793639,
0.0326487157876059,
0.0537639178458945,
0.0692898017122832,
0.0868118706472075,
0.452912212216652,
0.496872643392627,
0.63731535288116,
0.662999600762986,
0.679590116665927,
0.938473140220911,
0.951514882668677,
0.975380384154726,
0.979771991305505,
0.98469591447456,
0.995120436499135,
0.995652752517411,
0.997249700572239,
0.997338419908619,
0.998757929290689,
0.998846648627068,
0.998935367963447,
0.999024087299827,
0.999378964645344,
0.999645122654482,
0.999733841990861,
0.999778201659051,
0.999866920995431,
0.99991128066362,
0.99995564033181,
1,],
35:[0.000655522779416584,
0.0284169124877089,
0.0458865945591609,
0.0599147820386758,
0.071779744346116,
0.188364470665355,
0.204457554900032,
0.316027531956735,
0.342084562438544,
0.366502785971812,
0.863159619796787,
0.883906915765322,
0.917928548017043,
0.926057030481809,
0.938511963290724,
0.982005899705014,
0.985086856768272,
0.992133726647,
0.992723697148475,
0.99331366764995,
0.997967879383808,
0.99806620780072,
0.998164536217633,
0.998230088495575,
0.998328416912487,
0.998590626024254,
0.998623402163225,
0.99885283513602,
0.998918387413962,
0.99924614880367,
0.999278924942641,
0.999344477220583,
0.999410029498524,
0.999475581776466,
0.999541134054408,
0.999573910193379,
0.999672238610291,
0.999705014749262,
0.999737790888233,
0.999836119305145,
0.999868895444116,
0.999901671583087,
0.999934447722058,
0.999967223861028,
0.999999999999999,],
50:[0.000664147124401081,
0.030803776150793,
0.0469963155647622,
0.0619396258637865,
0.073024557630576,
0.0988630433751324,
0.106026344502601,
0.130062145195211,
0.137446828697481,
0.14842106927687,
0.560255538512626,
0.578314015085627,
0.602681889340438,
0.613197552143455,
0.633533104571546,
0.753490725659798,
0.765840699568304,
0.800091715555274,
0.809152579895317,
0.816616328531444,
0.967045652208289,
0.973402488970413,
0.97999652113411,
0.982653109631714,
0.985341324182861,
0.995477474343364,
0.996189060548079,
0.997517354796881,
0.99772292414491,
0.997817802305539,
0.998908901152769,
0.999019592340169,
0.99913028352757,
0.999146096554341,
0.999272600768513,
0.999414918009456,
0.999446544062999,
0.999557235250399,
0.999604674330713,
0.999620487357485,
0.99979443065197,
0.999841869732285,
0.999889308812599,
0.999920934866142,
0.999952560919685,
0.999968373946456,
0.999984186973228,
0.999999999999999,],
100:[0.000506234679739955,
0.0342108067782159,
0.0528349142065437,
0.0684749014174571,
0.0800117233294255,
0.104657359053607,
0.111344985612277,
0.127917510391132,
0.133779175103911,
0.142038793562826,
0.37080358094426,
0.380368751998294,
0.38833528722157,
0.393051262922306,
0.400218480230203,
0.423958222316956,
0.427661728658211,
0.43413620377278,
0.43767984653096,
0.442049451135031,
0.732894596610892,
0.746429713311307,
0.758952360652243,
0.765187040392198,
0.775551529361611,
0.846237877011616,
0.854710646914632,
0.877571139294468,
0.884125546200575,
0.890493445593093,
0.97071832036662,
0.975407652136843,
0.980976233613982,
0.98318767984653,
0.985772141106255,
0.995284024299264,
0.996376425450282,
0.997388894809762,
0.997628690184376,
0.997735265906426,
0.999227326015133,
0.999253969945646,
0.999307257806671,
0.999333901737184,
0.999360545667697,
0.99949376532026,
0.999520409250772,
0.999760204625386,
0.999786848555899,
0.999813492486411,
0.999840136416924,
0.999866780347437,
0.999893424277949,
0.999946712138975,
0.999973356069487,
1,],
150:[0.000811651711232357,
0.0381927221896559,
0.059926951345989,
0.076791270234928,
0.0894169635207647,
0.113991973666411,
0.121116472020561,
0.138521892050322,
0.144609279884565,
0.153131622852504,
0.367588041664787,
0.377553321008251,
0.385985480452721,
0.390043739008883,
0.39617621860486,
0.415250033818821,
0.418812282995896,
0.424493844974523,
0.427560084772512,
0.430040131667944,
0.593587951481264,
0.600126256932858,
0.604545249582901,
0.608197682283446,
0.611669747937051,
0.629841727916309,
0.632231591288271,
0.637372052126076,
0.639942282544979,
0.642557604725616,
0.756188844298146,
0.763674076746178,
0.773413897280966,
0.778779816927447,
0.785994498805068,
0.848942598187311,
0.854759435451143,
0.87117283672273,
0.875952563466654,
0.881408666636605,
0.972358750056364,
0.977273752085493,
0.982459304685034,
0.984488433963114,
0.986652838526401,
0.995310456779546,
0.996978851963746,
0.997835595436713,
0.998106146007124,
0.998286513054064,
0.999368715335708,
0.999458898859178,
0.999549082382648,
0.999684357667853,
0.999729449429589,
0.999774541191324,
0.999819632953059,
0.999864724714794,
0.999909816476529,
0.999954908238264,
1,],
200:[0.00128134090594265,
0.0432539132843884,
0.0662487879207646,
0.0861615182158193,
0.0997021748164565,
0.123805236182296,
0.131597174123839,
0.147873666712841,
0.154384263748441,
0.161864524172322,
0.352472641640116,
0.361164981299348,
0.367329269981991,
0.371484970217481,
0.377129796370688,
0.393475550630281,
0.396315279124532,
0.400678764371796,
0.402825876160132,
0.405457819642609,
0.52760077573071,
0.533072447707438,
0.536674054578196,
0.539202105554785,
0.542457404072586,
0.553608533037816,
0.555686383155561,
0.55887242000277,
0.5602576534146,
0.561989195179387,
0.605139215957888,
0.607944313616844,
0.610160687075772,
0.611407397146419,
0.613208200581797,
0.623112619476381,
0.624601745394098,
0.62678348801773,
0.628203352264856,
0.629969524864939,
0.733273306552153,
0.737567530128826,
0.743143094611441,
0.745878930599805,
0.749722953317633,
0.776077018977697,
0.7915223715196,
0.800110818672946,
0.802604238814239,
0.806517523202659,
0.857840421110956,
0.861442027981714,
0.866047929076048,
0.868541349217342,
0.871346446876298,
0.889493004571269,
0.892644410583182,
0.8973888350187,
0.898877960936417,
0.900713395207091,
0.937629865632358,
0.940469594126609,
0.94282449092672,
0.944278986009142,
0.946287574456295,
0.954252666574317,
0.955672530821442,
0.958269843468623,
0.959585815209862,
0.960313062751072,
0.972260700928105,
0.973438149328161,
0.974407812716442,
0.974858013575286,
0.975758415292976,
0.979706330516691,
0.980087269704944,
0.980814517246155,
0.981230087269704,
0.981611026457957,
0.990026319434824,
0.990615043634851,
0.991238398670175,
0.991480814517245,
0.991827122870202,
0.993177725446737,
0.99355866463499,
0.994078127164426,
0.994147388835018,
0.994355173846792,
0.995982823105692,
0.996086715611579,
0.996294500623354,
0.996363762293945,
0.996433023964537,
0.997021748164565,
0.997091009835156,
0.997333425682226,
0.997368056517522,
0.998510874082282,
0.998545504917578,
0.998614766588169,
0.998649397423465,
0.998753289929352,
0.998857182435239,
0.998891813270535,
0.998995705776422,
0.999030336611718,
0.999307383294084,
0.99934201412938,
0.999411275799971,
0.999445906635267,
0.999480537470563,
0.999515168305859,
0.999549799141154,
0.99958442997645,
0.999619060811746,
0.999653691647042,
0.999688322482337,
0.999722953317633,
0.999757584152929,
0.999792214988225,
0.99982684582352,
0.999861476658816,
0.999896107494112,
0.999930738329408,
0.999965369164703,
0.999999999999999,]}



total_redemption_rnd = {5:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
125,
135,
140,
145,
150,
165,
200,
215,
250,
325,],
10:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
125,
130,
135,
150,
155,
175,
195,
230,
515,],
15:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
110,
120,
125,
135,
150,
175,
200,
245,
250,],
25:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
100,
105,
110,
115,
125,
150,
155,
165,
175,
185,
230,
250,],
35:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
145,
150,
155,
160,
170,
175,
185,
190,
200,
210,
245,
250,
260,
275,
355,
525,
725,],
50:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185,
190,
195,
200,
210,
225,
230,
240,
275,
300,
380,],
100:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185,
190,
195,
200,
205,
210,
215,
220,
225,
230,
250,
260,
270,
275,
290,
295,
300,
305,
450,],
150:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185,
190,
195,
200,
205,
210,
215,
220,
225,
230,
235,
240,
245,
250,
255,
270,
275,
285,
290,
300,
310,
320,
490,
770,],
200:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185,
190,
195,
200,
205,
210,
215,
220,
225,
230,
235,
240,
245,
250,
255,
260,
265,
270,
275,
280,
285,
290,
295,
300,
305,
310,
315,
320,
325,
330,
335,
340,
345,
350,
355,
360,
365,
370,
375,
380,
385,
390,
395,
400,
405,
410,
415,
420,
425,
430,
435,
440,
445,
450,
455,
460,
465,
470,
475,
480,
485,
495,
500,
505,
510,
515,
520,
525,
530,
535,
540,
550,
555,
560,
585,
590,
595,
600,
610,
615,
620,
625,
645,
670,
750,
770,
775,
850,
1000,
1030,
1265,]}


In [ ]:
import pandas as pd
import random
import bisect

# Given values
values = [5, 10, 15, 25, 35, 50, 100, 150, 200]

# Create a DataFrame with a single column named 'Values'
df = pd.DataFrame({'Values': values})

values = df['Values'].to_list()

# create a list called "add_col"
add_col = []

# Iterate through each row in the "Values" column
for index, row in df.iterrows():
    value = row['Values']
    rand = random.random()
    
    dist_1 = points_balance_to_redeem[value] 
    dist_2 = total_redemption_rnd[value] 
    
    index = bisect.bisect_right(dist_1, rand) 
    
    if index == 0: 
        smallest_value = dist_1[0] 
        redemption_value = dist_2[0]
    elif index == len(dist_1): 
        smallest_value = dist_1[-1] 
        redemption_value = dist_2[-1]
    else: 
        smallest_value = dist_1[index] 
        redemption_value = dist_2[index]
    
    add_col.append(redemption_value)
    
df['redemption_value'] = add_col

df

In [ ]:
import pandas as pd
import random
import bisect

# Given values
values = [5, 10, 15, 25, 35, 50, 100, 150, 200]

# Create a DataFrame with a single column named 'Values'
df = pd.DataFrame({'Values': values})

values = df['Values'].to_list()

# create a list called "add_col"
add_col = []

# Iterate through each row in the "Values" column
for index, item in enumerate(values):
    value = row['Values']
    rand = random.random()
    
    dist_1 = points_balance_to_redeem[value] 
    dist_2 = total_redemption_rnd[value] 
    
    index = bisect.bisect_right(dist_1, rand) 
    
    if index == 0: 
        smallest_value = dist_1[0] 
        redemption_value = dist_2[0]
    elif index == len(dist_1): 
        smallest_value = dist_1[-1] 
        redemption_value = dist_2[-1]
    else: 
        smallest_value = dist_1[index] 
        redemption_value = dist_2[index]
    
    add_col.append(redemption_value)
    
df['redemption_value'] = add_col

df

In [ ]:

points_balance_to_redeem = {5:[0.00730897009966777,
0.382059800664452,
0.777408637873754,
1],
10:[0.00246949447995352,
0.0939860546194073,
0.256827425915166,
0.348634514816967,
0.427222545031958,
1],
15:[0.00120501725365613,
0.0500082160267295,
0.112450019170729,
0.157473845648245,
0.217505614284932,
0.794215917182451,
0.848003505504738,
1],
25:[0.00133815067576609,
0.0492439448681922,
0.0810919309514251,
0.104509567777332,
0.130938043623712,
0.68312591997859,
0.749431285962799,
0.961260537936572,
1],
35:[0.000741619697419163,
0.0321492138831207,
0.0519133788193414,
0.0677840403441115,
0.0812073568673984,
0.213104420053397,
0.231311183625037,
0.357534856125779,
0.38701423909819,
0.414639572827054,
0.976527736576684,
1],
50:[0.000677703553102914,
0.0314325362248685,
0.0479555942814729,
0.0632039242262884,
0.0745151192435538,
0.100881014619034,
0.108190531513215,
0.132716945815987,
0.140252363894536,
0.151450608319618,
0.571691354438958,
0.590118436763804,
0.614983702843128,
0.625714009100591,
0.646464646464647,
0.768870816794139,
0.781472875722077,
0.816423016103527,
0.825668828863717,
0.833284925936683,
0.986784780714493,
0.993271371865621,
1],
100:[0.000516051931120647,
0.0348742462925743,
0.0538595252322234,
0.069802813841056,
0.081563365745016,
0.106686946602205,
0.113504264218589,
0.130398174805802,
0.13637351295562,
0.144793307621272,
0.377994459231897,
0.387745124667282,
0.395866152425444,
0.400673583573252,
0.407979792492802,
0.432179911999565,
0.435955239285132,
0.442555271877886,
0.44616763539573,
0.45062197838014,
0.747107393122929,
0.760904992123418,
0.773670487261666,
0.780026074202836,
0.790591558476832,
0.862648704438047,
0.871285784127329,
0.894589602911619,
0.901271117388234,
0.907762507469173,
0.989543158237818,
0.994323428757673,
1],
150:[0.00093167701863354,
0.0438405797101449,
0.0687888198757764,
0.0881469979296066,
0.102639751552795,
0.130848861283644,
0.139026915113872,
0.159006211180124,
0.165993788819876,
0.175776397515528,
0.421946169772257,
0.433385093167702,
0.443064182194617,
0.447722567287785,
0.454761904761905,
0.476656314699793,
0.480745341614907,
0.487267080745342,
0.490786749482402,
0.493633540372671,
0.681366459627329,
0.688871635610766,
0.693944099378882,
0.698136645962733,
0.70212215320911,
0.722981366459627,
0.725724637681159,
0.731625258799172,
0.734575569358178,
0.737577639751553,
0.868012422360248,
0.876604554865424,
0.887784679089027,
0.893944099378882,
0.902225672877847,
0.974482401656315,
0.981159420289855,
1],
200:[0.00149368212829518,
0.0504218642767753,
0.0772274030115861,
0.100440030681038,
0.116224617496266,
0.144321989423116,
0.15340519155464,
0.172378991562714,
0.179968511565944,
0.188688385612208,
0.410883694642929,
0.421016511242986,
0.428202333373703,
0.43304670784385,
0.439626983165799,
0.458681522748375,
0.461991845302975,
0.467078438496629,
0.469581365306205,
0.472649469137298,
0.615033708772355,
0.621412135158048,
0.625610593032175,
0.628557587501514,
0.632352347503129,
0.645351418998022,
0.647773606233095,
0.651487626660207,
0.653102418150256,
0.655120907512817,
0.705421662427839,
0.708691615195188,
0.711275281579266,
0.71272859392031,
0.714827822857373,
0.726373582011223,
0.728109482863025,
0.730652779459852,
0.732307940737152,
0.734366799886964,
0.854789875257357,
0.859795728876509,
0.866295264623955,
0.869484477816802,
0.873965524201687,
0.904686932299867,
0.922691857413911,
0.932703564652214,
0.935610189334302,
0.94017197529369,
1]}


total_redemption_rnd = {5:[0,
5,
10,
15],
10:[0,
5,
10,
15,
20,
25],
15:[0,
5,
10,
15,
20,
25,
30,
35],
25:[0,
5,
10,
15,
20,
25,
30,
35,
40],
35:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55],
50:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110],
100:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160],
150:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185],
200:[0,
5,
10,
15,
20,
25,
30,
35,
40,
45,
50,
55,
60,
65,
70,
75,
80,
85,
90,
95,
100,
105,
110,
115,
120,
125,
130,
135,
140,
145,
150,
155,
160,
165,
170,
175,
180,
185,
190,
195,
200,
205,
210,
215,
220,
225,
230,
235,
240,
245,
250]}


In [ ]:
print(total_redemption_rnd)